## pipeline 준비

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

/opt/conda/envs/klue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

Downloading: 100%|██████████| 616/616 [00:00<00:00, 327kB/s]
Downloading: 100%|██████████| 2.24G/2.24G [00:36<00:00, 61.2MB/s]


In [3]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

Downloading: 100%|██████████| 5.07M/5.07M [00:01<00:00, 3.12MB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:01<00:00, 4.92MB/s]


In [5]:
task = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

## 데이터셋 준비

In [8]:
import pandas as pd

In [20]:
df = pd.read_csv("../dataset/train/train.csv")

In [93]:
sentences = df['sentence']

In [94]:
sentences = sentences.apply(lambda x: tokenizer(x, return_tensors="pt")["input_ids"])
sentences

0        [[tensor(0), tensor(6), tensor(90459), tensor(...
1        [[tensor(0), tensor(23646), tensor(13902), ten...
2        [[tensor(0), tensor(341), tensor(88774), tenso...
3        [[tensor(0), tensor(6), tensor(83654), tensor(...
4        [[tensor(0), tensor(28980), tensor(2680), tens...
                               ...                        
32465    [[tensor(0), tensor(9397), tensor(7641), tenso...
32466    [[tensor(0), tensor(37668), tensor(12284), ten...
32467    [[tensor(0), tensor(6), tensor(34433), tensor(...
32468    [[tensor(0), tensor(106752), tensor(88356), te...
32469    [[tensor(0), tensor(26191), tensor(46364), ten...
Name: sentence, Length: 32470, dtype: object

In [95]:
sentences[1]

tensor([[     0,  23646,  13902,    469,  96685,   1571,  11095,  44623,   5645,
          15960,   7641,   1280,   3032,   8825,   5301,   7641,   1280, 128682,
         219052,   7641,    469,  23526,   8177,  30679,  19625,  42482,    480,
          51638,   7641,   1963,  36192,  10047,   7641,    132,    713,  72047,
             16,   1654,  11105,  39191,  10047,   9514,      5,      2]])

In [29]:
import torch
from typing import Any, Optional, Tuple
from transformers import DataCollatorForLanguageModeling

In [96]:
def torch_mask_tokens(inputs: Any,tokenizer, mlm_probability:float=0.1,special_tokens_mask: Optional[Any] = None) -> Tuple[Any, Any]:
    """
    Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
    """
    import torch

    labels = inputs.clone()
    # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    if special_tokens_mask is None:
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
    else:
        special_tokens_mask = special_tokens_mask.bool()

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 1,dtype=torch.float32)).bool() & masked_indices
    print(indices_replaced)
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # # 10% of the time, we replace masked input tokens with random word
    # indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    # random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    # inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs

In [97]:
x = torch_mask_tokens(sentences[1], tokenizer=tokenizer, mlm_probability=0.1)
x

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False,  True, False, False, False,
         False,  True, False, False, False, False, False, False, False, False,
         False, False, False, False]])


tensor([[     0,  23646,  13902,    469,  96685,   1571,  11095,  44623,   5645,
          15960,   7641,   1280,   3032, 250001,   5301,   7641,   1280, 128682,
         219052,   7641,    469,  23526,   8177,  30679,  19625,  42482, 250001,
          51638,   7641,   1963,  36192, 250001,   7641,    132,    713,  72047,
             16,   1654,  11105,  39191,  10047,   9514,      5,      2]])

In [102]:
mask_sent = tokenizer.decode(x[0])
mask_sent

'<s> 호남이 기반인 바른미래당·대<mask>신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask>당(가칭)으로 재탄생한다.</s>'

In [103]:
task(mask_sent)

/opt/conda/envs/klue/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[[{'score': 0.11501485854387283,
   'token': 6775,
   'token_str': '우',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대우 신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask> 당(가칭)으로 재탄생한다.</s></s>'},
  {'score': 0.10313916951417923,
   'token': 5050,
   'token_str': '선',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대선 신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask> 당(가칭)으로 재탄생한다.</s></s>'},
  {'score': 0.10303967446088791,
   'token': 5102,
   'token_str': '치',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대치 신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask> 당(가칭)으로 재탄생한다.</s></s>'},
  {'score': 0.10192589461803436,
   'token': 3665,
   'token_str': '화',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대화 신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask> 당(가칭)으로 재탄생한다.</s></s>'},
  {'score': 0.07685703784227371,
   'token': 8825,
   'token_str': '안',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대안 신당·민주평화당이 우여곡절 끝<mask> 합당해 민<mask> 당(가칭)으로 재탄생한다.</s></s>'}],
 [{'score': 0.9984962940216064,
   'token': 480,
   'token_str': '에',
   'sequence': '<s><s> 호남이 기반인 바른미래당·대<mask> 신당·민주평화당이 우여곡절 끝에 합

### 내가 생각한 MASK토큰을 처리하는 방식?
##### 1. x%비율로 먼저 마스킹 처리를 하고,일괄적으로 MASK 토큰을 처리한다.
##### 2. x%비율로, 어느 인덱스(토큰의 위치)를 마스킹할지만 먼저 체크하고, for문을 돌면서 k개의 MASK토큰에 대해 k번 추측을 한다.

1.의 장점

    1. 2번의 방식에 비해 k배 빠르다.
    2. 단점 -> 마스킹한 토큰의 추측이 정확하지 않을 수 있다.하지만 반드시 안좋은 건 아닐것이다..

2.의 장점

    1. 1에비해 추측이 정확하다. 
    2. 속도가 느리다..

#### 추가로 생각해 볼 것 -> score기준을 어떤 식으로 할 것인가?
예를 들어 생각해보면.. score 를 0.90점 이상인 값들만 augment를 한다고 하면, 이 정도로 모델이 강하게 예측한 경우는 mask하기전 토큰과 동일할 확률이 높다. 이말인 즉슨, 버리는 mask가 많을 것이다.
반대로 score가 너무 낮으면, 문장의 의미가 변해서 augment한 데이터의 퀄리티가 떨어져 안하니만 못한 수준이 될 수 있다.

In [89]:
mask_sent

'<s> 호남이 기반인 바른<mask>래당·대안신당·민주평화당이 우여곡절 끝에<mask>당해 민생당(가<mask>)으로 재탄생한다<mask></s>'

In [92]:
task(mask_sent)[0][0]

/opt/conda/envs/klue/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'score': 0.8786227107048035,
 'token': 5645,
 'token_str': '미',
 'sequence': '<s><s> 호남이 기반인 바른미 래당·대안신당·민주평화당이 우여곡절 끝에<mask> 당해 민생당(가<mask> )으로 재탄생한다<mask></s></s>'}